In [1]:
import numpy as np
from pynq import Overlay, allocate
from time import time

In [33]:
# 1) Load your FPGA bitstream (make sure ntt.bit is on the board)
overlay = Overlay("intt.bit")

In [34]:
# overlay.download()

# 2) Grab the DMA IP (name it exactly as in your block design)
dma = overlay.axi_dma_0  # or adjust if yours is named differently

In [52]:
# 3) Prepare input data (16-point vector)
#    For example: a simple ramp 0,1,2,...,15
in_data = np.arange(512, dtype=np.uint16)
# in_data=[i+1 for i in in_data]
# 4) Allocate contiguous buffers in the PL-connected DDR
in_buf  = allocate(shape=(512,), dtype=np.uint32)
out_buf = allocate(shape=(512,), dtype=np.uint32)

# 5) Copy your data into the input buffer
in_buf[:] = in_data

In [53]:
# 5) Start
start_time=time()
dma.sendchannel.transfer(in_buf)
dma.recvchannel.transfer(out_buf)   # receive first

dma.sendchannel.wait()              # wait for TX to finish
dma.recvchannel.wait()              # then RX
end_time=time()

In [54]:
# 8) Inspect the result
print("Input: ", in_buf)
print("NTT Output:", out_buf[:])

# 9) (Optional) Clean up
# in_buf.freebuffer()
# out_buf.freebuffer()


Input:  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 24

In [55]:
hw_time=end_time-start_time
print(f'Hardware Time = {hw_time} sec')

Hardware Time = 0.008181095123291016 sec


In [56]:
import numpy as np
from time import time


In [57]:
MOD = 7681
ROOT_INV = 7480       # inverse of 7146 mod 7681
N = 512
N_INV = 7666      # inverse of 512 mod 7681

In [58]:
def mulmod(a, b):
    return (a * b) % MOD

def addmod(a, b):
    res = a + b
    return res - MOD if res >= MOD else res

def submod(a, b):
    res = a - b
    return res + MOD if res < 0 else res

def modpow(x, e):
    res = 1
    while e > 0:
        if e & 1:
            res = mulmod(res, x)
        x = mulmod(x, x)
        e >>= 1
    return res



In [59]:
def bit_reverse(x, logn):
    res = 0
    for i in range(logn):
        if x & (1 << i):
            res |= 1 << (logn - 1 - i)
    return res



In [60]:
def intt(a):
    len_ = N
    while len_ >= 2:
        wlen = modpow(ROOT_INV, N // len_)
        for i in range(0, N, len_):
            w = 1
            for j in range(len_ // 2):
                u = a[i + j]
                v = a[i + j + len_ // 2]
                a[i + j] = addmod(u, v)
                a[i + j + len_ // 2] = mulmod(submod(u, v), w)
                w = mulmod(w, wlen)
        len_ >>= 1

    # Final scaling: multiply by N⁻¹ mod MOD
    for i in range(N):
        a[i] = mulmod(a[i], N_INV)

    return a

In [61]:
input_data = [i % MOD for i in range(N)]

start_time = time()
output_data = intt(input_data.copy())
end_time = time()

print("First 8 output values:", output_data[:8])
print("Time taken: {:.6f} seconds".format(end_time - start_time))
sw_time=end_time-start_time

First 8 output values: [4096, 3840, 1691, 5989, 2047, 1335, 6345, 5633]
Time taken: 0.046005 seconds


In [62]:
speedup=sw_time/hw_time
print(speedup)

5.623331584775893


In [63]:
count=0
error_idx=[]
for i in range(512):
    if out_buf[i]!=output_data[i]:
        count+=1
        error_idx.append(i)
#         break
print("Done")
print(count)

Done
480


In [64]:
error_idx

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 125,
 126,
 127,
 128,
 129,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,

In [65]:
out_buf[1]

3080

In [66]:
output_data[1]

3840

In [67]:
exp=[4096, 3080, 3057, 5168, 2817, 2901, 7187, 4391, 3611, 1437, 439, 7105, 5317, 547, 566, 1510, 4585, 5870, 1656, 2108, 4903, 52, 3302, 7006, 1377, 5500, 6874, 2636, 1842, 1569, 3071, 1108, 6299, 6654, 6141, 122, 2242, 7378, 1041, 5619, 559, 473, 3025, 912, 3687, 888, 4428, 3234, 3493, 7614, 3744, 4762, 5382, 4932, 1427, 2931, 3372, 3652, 5409, 6902, 3242, 5871, 1361, 3949, 2047, 7149, 5869, 4191, 4234, 2187, 4638, 4000, 5070, 703, 3073, 6880, 535, 2240, 2935, 614, 3526, 4063, 5046, 6775, 677, 2889, 2880, 23, 2339, 3710, 5112, 988, 527, 3260, 1105, 1147, 2972, 5795, 24, 7074, 3802, 4865, 3067, 6324, 1532, 4741, 5398, 1295, 420, 6563, 484, 5072, 3805, 524, 1790, 5555, 6937, 6592, 6645, 1313, 1693, 3570, 7595, 6836, 3659, 5410, 468, 997, 1691, 6152, 1598, 4624, 7097, 2528, 2470, 2126, 2358, 6970, 6757, 7056, 2794, 5655, 3914, 7351, 7077, 2046, 919, 6622, 5726, 5635, 7381, 5675, 6435, 6364, 6113, 3692, 4598, 962, 313, 2348, 5643, 2436, 2070, 5161, 3334, 2385, 6740, 871, 2498, 3268, 3838, 2893, 3361, 975, 6058, 7603, 7601, 7668, 1419, 5375, 3026, 7479, 6982, 6545, 6277, 3852, 6730, 4034, 7198, 6931, 7036, 3952, 6345, 3012, 1723, 6042, 7536, 1292, 6896, 71, 3442, 1962, 586, 1816, 2317, 6308, 3820, 7565, 5229, 1023, 3373, 6025, 6689, 2832, 1670, 5742, 1187, 7429, 7651, 1231, 7463, 6207, 5353, 2596, 2204, 6066, 4267, 6112, 2778, 7462, 3790, 5692, 2071, 5830, 3608, 1682, 3784, 25, 4852, 1268, 7348, 3048, 4562, 2160, 2562, 500, 5994, 5805, 2121, 3258, 4233, 2022, 3275, 6155, 5103, 4646, 3840, 3034, 2577, 1525, 4405, 5658, 3447, 4422, 5559, 1875, 1686, 7180, 5118, 5520, 3118, 4632, 332, 6412, 2828, 7655, 3896, 5998, 4072, 1850, 5609, 1988, 3890, 218, 4902, 1568, 3413, 1614, 5476, 5084, 2327, 1473, 217, 6449, 29, 251, 6493, 1938, 6010, 4848, 991, 1655, 4307, 6657, 2451, 115, 3860, 1372, 5363, 5864, 7094, 5718, 4238, 7609, 784, 6388, 144, 1638, 5957, 4668, 1335, 3728, 644, 749, 482, 3646, 950, 3828, 1403, 1135, 698, 201, 4654, 2305, 6261, 12, 79, 77, 1622, 6705, 4319, 4787, 3842, 4412, 5182, 6809, 940, 5295, 4346, 2519, 5610, 5244, 2037, 5332, 7367, 6718, 3082, 3988, 1567, 1316, 1245, 2005, 299, 2045, 1954, 1058, 6761, 5634, 603, 329, 3766, 2025, 4886, 624, 923, 710, 5322, 5554, 5210, 5152, 583, 3056, 6082, 1528, 5989, 6683, 7212, 2270, 4021, 844, 85, 4110, 5987, 6367, 1035, 1088, 743, 2125, 5890, 7156, 3875, 2608, 7196, 1117, 7260, 6385, 2282, 2939, 6148, 1356, 4613, 2815, 3878, 606, 7656, 1885, 4708, 6533, 6575, 4420, 7153, 6692, 2568, 3970, 5341, 7657, 4800, 4791, 7003, 905, 2634, 3617, 4154, 7066, 4745, 5440, 7145, 800, 4607, 6977, 2610, 3680, 3042, 5493, 3446, 3489, 1811, 531, 5633, 3731, 6319, 1809, 4438, 778, 2271, 4028, 4308, 4749, 6253, 2748, 2298, 2918, 3936, 66, 4187, 4446, 3252, 6792, 3993, 6768, 4655, 7207, 7121, 2061, 6639, 302, 5438, 7558, 1539, 1026, 1381, 6572, 4609, 6111, 5838, 5044, 806, 2180, 6303, 674, 4378, 7628, 2777, 5572, 6024, 1810, 3095, 6170, 7114, 7133, 2363, 575, 7241, 6243, 4069, 3289, 493, 4779, 4863, 2512, 4623, 4600]

In [68]:
for i in range(512):
    if out_buf[i]!=exp[i]:
        print(i)